## Execute from VSCode

In [1]:
import pandas as pd
from datetime import datetime
import holidays_co
import io
from shareplum import Site
from shareplum import Office365
from shareplum.site import Version
from pathlib import Path

# Configurar el año y mes manualmente si es necesario
anio = None  # Cambiar este valor al año deseado
mes = None   # Cambiar este valor al mes deseado

# Configurar el año y mes (puede cambiarse manualmente. si se deja None, tomará el año y mes actual)
anio = datetime.now().year if anio is None else anio
mes = datetime.now().month if mes is None else mes

# Función para obtener el nombre del día en español
def obtener_dia_semana(fecha):
    dias_semana = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
    dia_semana = datetime.strptime(fecha, "%d-%m-%Y").weekday()
    return dias_semana[dia_semana]

# Función para determinar el tipo de día
def tipo_dia(fecha):
    fecha_obj = datetime.strptime(fecha, "%d-%m-%Y").date()
    if holidays_co.is_holiday_date(fecha_obj):  # Verificar si es festivo
        return "Festivo"
    dia_semana = fecha_obj.weekday()  # 0: Lunes, 1: Martes, ..., 6: Domingo
    return ["Hábil", "Hábil", "Hábil", "Hábil", "Hábil", "Sábado", "Domingo"][dia_semana]

# Función para mapear el NT por rangos
def mapear_nt_por_rango(valor):
    if valor in {1, 2}:  # Comprobamos si el valor es exactamente 1 o 2
        return valor
    elif valor < 1:
        return 1
    elif valor < 30:
        return 2
    elif valor < 57.5:
        return 3
    return None  # O un valor por defecto si no encaja en ningún rango

# Función para conectarse a SharePoint
def conectar_sharepoint(sharepoint_url, sharepoint_site, sharepoint_user, sharepoint_password):
    authcookie = Office365(sharepoint_url, username=sharepoint_user, password=sharepoint_password).GetCookies()
    site = Site(f"{sharepoint_url}/sites/{sharepoint_site}", authcookie=authcookie, version=Version.v365)
    return site

# Función para cargar archivos desde SharePoint
def cargar_archivos_desde_sharepoint(folder, valid_extensions):
    files = folder.files
    aenc_files = [file for file in files if 'aenc' in file['Name'] and file['Name'].endswith(valid_extensions)]
    tfroc_files = [file for file in files if 'tfroc' in file['Name'] and file['Name'].endswith(valid_extensions)]
    return aenc_files, tfroc_files

# Función para procesar los archivos
def procesar_archivos():
    # 🔹 DATOS DE SHAREPOINT
    sharepoint_url = "https://ruitoqueesp1.sharepoint.com"
    sharepoint_site = "fronterascomerciales"  # Nombre del sitio en SharePoint
    sharepoint_user = "rbadillo@ruitoqueesp.com"
    sharepoint_password = "r2083502R"
    sharepoint_folder = f"Documentos Compartidos/aenc/{anio}/{mes:02d}"

    # Conectarse a SharePoint
    site = conectar_sharepoint(sharepoint_url, sharepoint_site, sharepoint_user, sharepoint_password)
    folder = site.Folder(sharepoint_folder)

    # Cargar archivos "aenc" y "tfroc" desde SharePoint
    valid_extensions = ('.TxF', '.TxR', '.Tx2')
    aenc_files, tfroc_files = cargar_archivos_desde_sharepoint(folder, valid_extensions)

    # Contar la cantidad de archivos "aenc" y "tfroc"
    num_aenc_files = len(aenc_files)
    num_tfroc_files = len(tfroc_files)
    print(f"Cantidad de archivos 'aenc' cargados: {num_aenc_files}")
    print(f"Cantidad de archivos 'tfroc' cargados: {num_tfroc_files}")

    # Asegurarse de que la cantidad de archivos "aenc" sea igual a la cantidad de archivos "tfroc"
    if num_aenc_files != num_tfroc_files:
        print("La cantidad de archivos 'aenc' no es igual a la cantidad de archivos 'tfroc'. El proceso no puede continuar.")
        return

    # Inicializar dataframes vacíos para consolidar los resultados
    consolidated_aenc_df = pd.DataFrame()
    consolidated_df = pd.DataFrame()

    # Diccionario de mapeo para la columna "OR"
    mapeo_or = {
        "CUNM": "ENEL",
        "SOLM": "AIRE",
        "SANM": "ESSA",
        "MARM": "AFINIA",
        "NSAM": "ESSA",
        "BOYM": "EBSA",
        "CASM": "ENERCA",
        "METM": "EMSA",
        "QUIM": "EDEQ",
        "RUIM": "RUITOQUE",
        "CHOM": "DISPAC",
        "CLOM": "EMCALI"
    }

    # Procesar archivos día a día
    for aenc_file in aenc_files:
        aenc_file_name = aenc_file['Name']
        aenc_file_content = folder.get_file(aenc_file_name)
        # Extraer el día y el mes del nombre del archivo
        day = Path(aenc_file_name).stem[-4:]  # Los últimos 4 caracteres son el día
        date = f"{day[2:]}-{day[:2]}-{anio}"  # Formatear como dd-mm-aaaa

        # Buscar el archivo tfroc correspondiente
        tfroc_file = next((f for f in tfroc_files if day in f['Name']), None)
        if tfroc_file is None:
            print(f"No se encontró un archivo tfroc correspondiente para el día {day}.")
            continue

        tfroc_file_content = folder.get_file(tfroc_file['Name'])

        # Leer los archivos con el separador ";"
        aenc_df = pd.read_csv(io.BytesIO(aenc_file_content), sep=";", encoding='latin1')
        tfroc_df = pd.read_csv(io.BytesIO(tfroc_file_content), sep=";", encoding='latin1')

        # Renombrar columnas para facilitar la unión
        aenc_df.rename(columns={"CODIGO SIC": "CODIGO FRONTERA"}, inplace=True)

        # Agregar columnas adicionales al dataframe de aenc
        horas = [col for col in aenc_df.columns if "HORA" in col]
        aenc_df.insert(0, "FECHA", date)
        aenc_df.insert(1, "DIA", obtener_dia_semana(date))
        aenc_df.insert(2, "TIPO_DIA", tipo_dia(date))
        aenc_df["TOTAL CONSUMO"] = aenc_df.loc[:, horas].sum(axis=1)

        # Consolidar los datos de aenc
        consolidated_aenc_df = pd.concat([consolidated_aenc_df, aenc_df], ignore_index=True)

        # Unir los dos dataframes utilizando "CODIGO FRONTERA"
        merged_df = pd.merge(aenc_df, tfroc_df[["CODIGO FRONTERA", "FACTOR DE PERDIDAS", "MERCADO COMERCIALIZACIÓN QUE EXPORTA", "NIVEL DE TENSION"]], on="CODIGO FRONTERA", how="inner")

        # Dividir los valores de consumo horario por el FACTOR DE PERDIDAS
        for hora in horas:
            merged_df[hora] = merged_df[hora] / merged_df["FACTOR DE PERDIDAS"]

        # Agregar columnas adicionales al archivo de salida
        merged_df.rename(columns={"CODIGO PROPIO": "NOMBRE FRONTERA"}, inplace=True)
        daily_df = merged_df.loc[:, ["CODIGO FRONTERA", "NOMBRE FRONTERA", "MERCADO COMERCIALIZACIÓN QUE EXPORTA", "NIVEL DE TENSION","TIPO DE AGRUPACIÓN", "IMPO - EXPO"] + horas]

        # Agregar columna de fecha, día y tipo de día
        daily_df.insert(0, "FECHA", date)
        daily_df.insert(1, "DIA", obtener_dia_semana(date))
        daily_df.insert(2, "TIPO_DIA", tipo_dia(date))

        # Agregar columna "OR" basada en el mapeo
        daily_df.insert(6, "OR", daily_df["MERCADO COMERCIALIZACIÓN QUE EXPORTA"].map(mapeo_or))

        # Agregar columna "NT" basada en la función de mapeo por rangos
        daily_df.insert(8, "NT", daily_df["NIVEL DE TENSION"].map(mapear_nt_por_rango))

        # Agregar columna de total de consumo por frontera
        daily_df["TOTAL CONSUMO"] = daily_df.loc[:, horas].sum(axis=1)

        # Agregar los resultados al dataframe consolidado
        consolidated_df = pd.concat([consolidated_df, daily_df], ignore_index=True)
        
    # Ordenar los DataFrames por las columnas "FECHA" y "CODIGO FRONTERA"
    consolidated_aenc_df = consolidated_aenc_df.sort_values(by=["FECHA", "CODIGO FRONTERA"])
    consolidated_df = consolidated_df.sort_values(by=["FECHA", "CODIGO FRONTERA"])

    # Convertir los dataframes a CSV en memoria y subir a SharePoint
    output_aenc_file_name = f"aenc_consolidado_{mes:02d}_{anio}.csv"
    output_file_name = f"consumos_{mes:02d}_{anio}.csv"
    output_total_file_name = f"total_consumo_{mes:02d}_{anio}.csv"

    # Subir el archivo de aenc consolidado a SharePoint
    aenc_csv = consolidated_aenc_df.to_csv(index=False, sep=",", encoding='utf-8-sig')
    folder.upload_file(aenc_csv.encode('utf-8-sig'), output_aenc_file_name)
    print(f"Archivo subido a SharePoint: {output_aenc_file_name}")

    # Subir el archivo de consumos consolidado a SharePoint
    consumos_csv = consolidated_df.to_csv(index=False, sep=",", encoding='utf-8-sig')
    folder.upload_file(consumos_csv.encode('utf-8-sig'), output_file_name)
    print(f"Archivo subido a SharePoint: {output_file_name}")

    # Generar y subir el archivo consolidado total de consumo por frontera a SharePoint
    consolidated_total_df = consolidated_df.groupby(["CODIGO FRONTERA", "NOMBRE FRONTERA","TIPO DE AGRUPACIÓN", "IMPO - EXPO"], as_index=False).agg({"TOTAL CONSUMO": "sum"})
    total_consumo_csv = consolidated_total_df.to_csv(index=False, sep=",", encoding='utf-8-sig')
    folder.upload_file(total_consumo_csv.encode('utf-8-sig'), output_total_file_name)
    print(f"Archivo subido a SharePoint: {output_total_file_name}")

    # Mensaje de confirmación al finalizar el proceso
    print("El proceso ha finalizado exitosamente.")

# Ejecutar la función automáticamente
procesar_archivos()

ShareplumRequestError: Shareplum HTTP Post Failed : 403 Client Error: Forbidden for url: https://ruitoqueesp1.sharepoint.com/_forms/default.aspx?wa=wsignin1.0

## Actualizar `consumos_{anio}` en la carpeta *fact_consumos*

In [2]:
import pandas as pd
from datetime import datetime
from shareplum import Office365, Site
from shareplum.site import Version
import io
import os

# Configurar el año y mes manualmente si es necesario
anio = None  # Cambiar este valor al año deseado
mes = None   # Cambiar este valor al mes deseado

# Configurar el año y mes (puede cambiarse manualmente. si se deja None, tomará el año y mes actual)
anio = datetime.now().year if anio is None else anio
mes = datetime.now().month if mes is None else mes

# Nombres de los archivos
output_file_name = f"consumos_{mes:02d}_{anio}.csv"
consumos_anio_file_name = f"consumos_{anio}.csv"

def conectar_sharepoint(sharepoint_url, sharepoint_site, sharepoint_user, sharepoint_password):
    """Función para conectarse a SharePoint"""
    authcookie = Office365(sharepoint_url, username=sharepoint_user, password=sharepoint_password).GetCookies()
    site = Site(f"{sharepoint_url}/sites/{sharepoint_site}", authcookie=authcookie, version=Version.v365)
    return site

def cargar_archivo_desde_sharepoint(folder, file_name):
    """Función para cargar archivos desde SharePoint"""
    print(f"Cargando archivo '{file_name}' desde SharePoint...")
    file_content = folder.get_file(file_name)
    df = pd.read_csv(io.BytesIO(file_content), sep=",", encoding='utf-8-sig')
    print(f"Archivo '{file_name}' cargado exitosamente.")
    return df

def subir_archivo_a_sharepoint(folder, file_name, df):
    """Función para guardar localmente y subir a SharePoint sin líneas vacías"""
    print(f"Preparando para guardar y subir '{file_name}'...")
    # Crear carpeta local si no existe
    os.makedirs("archivos_descargados", exist_ok=True)

    # 1) Eliminar filas completamente vacías
    df_clean = df.dropna(how="all")

    # 2) Guardar CSV local sin líneas vacías adicionales
    local_path = os.path.join("archivos_descargados", file_name)
    with open(local_path, "w", newline="", encoding="utf-8-sig") as f:
        # Escribir con configuración que evita líneas en blanco extra
        df_clean.to_csv(f, index=False, sep=",", encoding="utf-8-sig")
    print(f"Archivo guardado en: {local_path}")

    # 3) Leer binario y subir a SharePoint
    try:
        with open(local_path, "rb") as fbin:
            file_bytes = fbin.read()
        print(f"Subiendo '{file_name}' a SharePoint...")
        folder.upload_file(file_bytes, file_name)
        print(f"Archivo '{file_name}' subido exitosamente.")
    except Exception as e:
        print(f"Error al subir '{file_name}': {e}")

def procesar_archivos(anio, mes, sharepoint_url, sharepoint_site, sharepoint_user, sharepoint_password):
    """Función principal para procesar los archivos"""
    site = conectar_sharepoint(sharepoint_url, sharepoint_site, sharepoint_user, sharepoint_password)

    # Ruta del SharePoint para el archivo "consumos_{anio}.csv"
    sharepoint_folder_anio = f"Documentos Compartidos/aenc/fact_consumos"
    folder_anio = site.Folder(sharepoint_folder_anio)

    # Ruta del SharePoint para el archivo "consumos_{mes:02d}_{anio}.csv"
    sharepoint_folder_mes = f"Documentos Compartidos/aenc/{anio}/{mes:02d}"
    folder_mes = site.Folder(sharepoint_folder_mes)

    # Cargar los archivos desde SharePoint
    consumos_anio_df = cargar_archivo_desde_sharepoint(folder_anio, consumos_anio_file_name)
    consumos_mes_df = cargar_archivo_desde_sharepoint(folder_mes, output_file_name)

    # Convertir la columna "FECHA" a datetime para facilitar el filtrado
    consumos_anio_df['FECHA'] = pd.to_datetime(consumos_anio_df['FECHA'], format='%Y-%m-%d')
    consumos_mes_df['FECHA'] = pd.to_datetime(consumos_mes_df['FECHA'], format='%d-%m-%Y')

    # Filtrar los datos del mes actual en el archivo "consumos_{anio}.csv"
    consumos_anio_df = consumos_anio_df[~((consumos_anio_df['FECHA'].dt.year == anio) & (consumos_anio_df['FECHA'].dt.month == mes))]

    # Concatenar los datos del archivo "consumos_{mes:02d}_{anio}.csv" con el archivo "consumos_{anio}.csv"
    print("Concatenando los datos...")
    consumos_actualizado_df = pd.concat([consumos_anio_df, consumos_mes_df], ignore_index=True)
    print("Concatenación completada.")

    # Ordenar el DataFrame resultante por las columnas "FECHA" y "CODIGO FRONTERA"
    consumos_actualizado_df = consumos_actualizado_df.sort_values(by=["FECHA", "CODIGO FRONTERA"])

    # Guardar el DataFrame resultante de nuevo en "consumos_{anio}.csv"
    subir_archivo_a_sharepoint(folder_anio, consumos_anio_file_name, consumos_actualizado_df)

# Datos de SharePoint
sharepoint_url = "https://ruitoqueesp1.sharepoint.com"
sharepoint_site = "fronterascomerciales"  # Nombre del sitio en SharePoint
sharepoint_user = "rbadillo@ruitoqueesp.com"
sharepoint_password = "r2083502R"

# Ejecutar la función principal
procesar_archivos(anio, mes, sharepoint_url, sharepoint_site, sharepoint_user, sharepoint_password)

Cargando archivo 'consumos_2025.csv' desde SharePoint...
Archivo 'consumos_2025.csv' cargado exitosamente.
Cargando archivo 'consumos_08_2025.csv' desde SharePoint...
Archivo 'consumos_08_2025.csv' cargado exitosamente.
Concatenando los datos...
Concatenación completada.
Preparando para guardar y subir 'consumos_2025.csv'...
Archivo guardado en: archivos_descargados\consumos_2025.csv
Subiendo 'consumos_2025.csv' a SharePoint...
Error al subir 'consumos_2025.csv': Shareplum HTTP Post Failed : ('Connection aborted.', TimeoutError('The write operation timed out'))


## Borrar la carpeta `content`

Ejecuta el siguiente código para borrar rapidamente la carpeta "content" y todos sus archivos temporales

In [ ]:
# Ruta de la carpeta predeterminada en Google Colab
folder_path = '/content'

# Eliminar todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)

print("\nTodos los archivos cargados han sido eliminados de la carpeta content de Google Colab.")